In [1]:
# Importando as bibliotecas
import re
import mysql.connector
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, to_timestamp, month, hour, datediff, dayofweek
from pyspark.sql.types import StringType
from pyspark.ml.feature import Tokenizer

## Carregando os dados
Nesta etapa, irei carregar os dados.

In [2]:
# Iniciando sessão no Spark
spark = SparkSession.builder.appName('Cluster').getOrCreate()


23/06/01 16:09:33 WARN Utils: Your hostname, daniel-VJFE43F11X-XXXXXX resolves to a loopback address: 127.0.1.1; using 192.168.0.157 instead (on interface wlo1)
23/06/01 16:09:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/01 16:09:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Lendos os dados csv
customers = spark.read.csv("../data/raw/olist_customers_dataset.csv", header=True)
geolocalization = spark.read.csv("../data/raw/olist_geolocation_dataset.csv", header=True)
order_items = spark.read.csv("../data/raw/olist_order_items_dataset.csv", header=True)
order_payments = spark.read.csv("../data/raw/olist_order_payments_dataset.csv", header=True)
order_reviews = spark.read.csv("../data/raw/olist_order_reviews_dataset.csv", header=True)
orders = spark.read.csv("../data/raw/olist_orders_dataset.csv", header=True)
products = spark.read.csv("../data/raw/olist_products_dataset.csv", header=True)
sellers = spark.read.csv("../data/raw/olist_sellers_dataset.csv", header=True)
product_category = spark.read.csv("../data/raw/product_category_name_translation.csv", header=True)

# Lendo os dados externos
ceps = spark.read.parquet("../data/external/cep_prefixes_processed.parquet")

## Transformando os dados
Nesta etapa, irei realizar uma série de transformações nas tabelas de forma individual, realizando tratamento específico para cada uma. Além disso, também irei gerar metadados, especificando o schema e descrição de cada coluna da tabela.

### Criando tabelas temporárias

In [4]:
# Criando tabelas temporárias
customers.createOrReplaceTempView("customers")
geolocalization.createOrReplaceTempView('geolocalization')
order_items.createOrReplaceTempView('order_items')
order_payments.createOrReplaceTempView('order_payments')
order_reviews.createOrReplaceTempView('order_reviews')
orders.createOrReplaceTempView('orders')
products.createOrReplaceTempView('products')
sellers.createOrReplaceTempView('sellers')
product_category.createOrReplaceTempView('product_category')
ceps.createOrReplaceTempView('ceps')

## customers

In [5]:
# Antes de decidir as mudanças, devemos checar as primeiras linhas
spark.sql("SELECT *  FROM customers LIMIT 5").show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                   09790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                   01151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                   08775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+



In [6]:
# Agora veremos o schema para verificar se o dado condiz com o seu tipo
customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [7]:
# Verificando se há inconsistencias nos valores unicos da coluna customer_state
customers.select('customer_state').distinct().collect()

[Row(customer_state='SC'),
 Row(customer_state='RO'),
 Row(customer_state='PI'),
 Row(customer_state='AM'),
 Row(customer_state='RR'),
 Row(customer_state='GO'),
 Row(customer_state='TO'),
 Row(customer_state='MT'),
 Row(customer_state='SP'),
 Row(customer_state='ES'),
 Row(customer_state='PB'),
 Row(customer_state='RS'),
 Row(customer_state='MS'),
 Row(customer_state='AL'),
 Row(customer_state='MG'),
 Row(customer_state='PA'),
 Row(customer_state='BA'),
 Row(customer_state='SE'),
 Row(customer_state='PE'),
 Row(customer_state='CE'),
 Row(customer_state='RN'),
 Row(customer_state='RJ'),
 Row(customer_state='MA'),
 Row(customer_state='AC'),
 Row(customer_state='DF'),
 Row(customer_state='PR'),
 Row(customer_state='AP')]

In [8]:
customers.select('customer_city').collect()

[Row(customer_city='franca'),
 Row(customer_city='sao bernardo do campo'),
 Row(customer_city='sao paulo'),
 Row(customer_city='mogi das cruzes'),
 Row(customer_city='campinas'),
 Row(customer_city='jaragua do sul'),
 Row(customer_city='sao paulo'),
 Row(customer_city='timoteo'),
 Row(customer_city='curitiba'),
 Row(customer_city='belo horizonte'),
 Row(customer_city='montes claros'),
 Row(customer_city='rio de janeiro'),
 Row(customer_city='lencois paulista'),
 Row(customer_city='sao paulo'),
 Row(customer_city='caxias do sul'),
 Row(customer_city='piracicaba'),
 Row(customer_city='rio de janeiro'),
 Row(customer_city='guarulhos'),
 Row(customer_city='sao paulo'),
 Row(customer_city='pacaja'),
 Row(customer_city='florianopolis'),
 Row(customer_city='aparecida de goiania'),
 Row(customer_city='sao paulo'),
 Row(customer_city='curitiba'),
 Row(customer_city='sao paulo'),
 Row(customer_city='santo andre'),
 Row(customer_city='goiania'),
 Row(customer_city='sao paulo'),
 Row(customer_city

Aparentemente, as colunas ``customer_state`` e ``customer_city`` necessitarão de
mudanças, onde a primeira receberá uma normalização de case e a segunda será 
substituída pela coluna de cidades da tabela externa, que não apresenta erros. 
Além disso, vale ressaltar que a coluna ``customer_zip_code_prefix``, mesmo 
apresentando valores numéricos, será mantida como **string**, pois ela perderia 
os zeros no início caso fosse alterada para formatos numéricos como **Int**. 


In [9]:
customers.columns

['customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state']

In [10]:
# Tratando a tabela customers
customers = spark.sql("SELECT cs.customer_id,  \
                        cs.customer_unique_id, \
                        cs.customer_zip_code_prefix, \
                        LOWER(cp.locality) AS customer_city,  \
                        LOWER(cs.customer_state) AS customer_state \
                       FROM customers cs  \
                        LEFT JOIN ceps cp  \
                        ON CAST(cs.customer_zip_code_prefix AS INT) = cp.cep_prefix")

In [11]:
# Verificando o resultado
customers.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            sp|
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            sp|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                   09790|são bernardo do c...|            sp|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                   09790|são bernardo do c...|            sp|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                   01151|           são paulo|            sp|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                   01151|           são paulo|            sp|
|b2b6027bc5c5109e5...|259dac757896d24d7...|            

## geolocalization

In [12]:
# Irei verificar as primeiras linhas e o schema novamente
geolocalization.show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                      01037| -23.54562128115268| -46.63929204800168|       sao paulo|               SP|
|                      01046|-23.546081127035535| -46.64482029837157|       sao paulo|               SP|
|                      01046| -23.54612896641469| -46.64295148361138|       sao paulo|               SP|
|                      01041|  -23.5443921648681| -46.63949930627844|       sao paulo|               SP|
|                      01035|-23.541577961711493| -46.64160722329613|       sao paulo|               SP|
|                      01012|-23.547762303364266| -46.63536053788448|       são paulo|               SP|
|                      01047|-23.546273112412678| -46.6

In [13]:
geolocalization.printSchema()

root
 |-- geolocation_zip_code_prefix: string (nullable = true)
 |-- geolocation_lat: string (nullable = true)
 |-- geolocation_lng: string (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [14]:
# Verificando se há inconsistencias nos valores unicos da coluna geolocation_state
geolocalization.select('geolocation_state').distinct().collect()

[Row(geolocation_state='SP'),
 Row(geolocation_state='RN'),
 Row(geolocation_state='AC'),
 Row(geolocation_state='ES'),
 Row(geolocation_state='RJ'),
 Row(geolocation_state='MG'),
 Row(geolocation_state='PI'),
 Row(geolocation_state='AM'),
 Row(geolocation_state='RR'),
 Row(geolocation_state='GO'),
 Row(geolocation_state='PB'),
 Row(geolocation_state='AL'),
 Row(geolocation_state='PA'),
 Row(geolocation_state='BA'),
 Row(geolocation_state='SE'),
 Row(geolocation_state='PE'),
 Row(geolocation_state='CE'),
 Row(geolocation_state='MA'),
 Row(geolocation_state='DF'),
 Row(geolocation_state='AP'),
 Row(geolocation_state='SC'),
 Row(geolocation_state='RO'),
 Row(geolocation_state='TO'),
 Row(geolocation_state='MT'),
 Row(geolocation_state='RS'),
 Row(geolocation_state='MS'),
 Row(geolocation_state='PR')]

In [15]:
# Verificando se há inconsistência nos valores únicos da coluna geolocation_city
geolocalization.select('geolocation_city').distinct().collect()

[Row(geolocation_city='sao bernardo do campo'),
 Row(geolocation_city='são paulo'),
 Row(geolocation_city='sãopaulo'),
 Row(geolocation_city='sao jose dos campos'),
 Row(geolocation_city='jundiaí'),
 Row(geolocation_city='sp'),
 Row(geolocation_city='sao paulo'),
 Row(geolocation_city='taboão da serra'),
 Row(geolocation_city='sa£o paulo'),
 Row(geolocation_city='redencao da serra'),
 Row(geolocation_city='itanhaem'),
 Row(geolocation_city='carapicuiba'),
 Row(geolocation_city='sao francisco da praia'),
 Row(geolocation_city='taubaté'),
 Row(geolocation_city='iguape'),
 Row(geolocation_city='itapecerica da serra'),
 Row(geolocation_city='santos'),
 Row(geolocation_city='franco da rocha'),
 Row(geolocation_city='cotia'),
 Row(geolocation_city='suzano'),
 Row(geolocation_city='embu guaçu'),
 Row(geolocation_city='caruara'),
 Row(geolocation_city='mogi das cruzes'),
 Row(geolocation_city='cananeia'),
 Row(geolocation_city='salesopolis'),
 Row(geolocation_city='jordanesia'),
 Row(geolocati

Aqui temos uma situação semelhante a da tabela anterior, sendo necessário 
normalizar a coluna ``geolocation_state``, mas com adições, onde as colunas que
representam dados geográficos também terão seu tipo alterado. Além disso, a 
coluna ``geolocation_city``apresentou alguns valores duplicados, mas que se 
referem a mesma cidade, então será necessário realizar uma limpeza nela.

In [16]:
geolocalization = spark.sql("SELECT g.geolocation_zip_code_prefix,  \
                          CAST(g.geolocation_lat AS DOUBLE),  \
                          CAST(g.geolocation_lng AS DOUBLE),  \
                          LOWER(cp.locality) AS geolocation_city, \
                          g.geolocation_city AS geolocation_city_original, \
                          LOWER(geolocation_state) AS geolocation_state \
                         FROM geolocalization AS g  \
                         LEFT JOIN ceps cp  \
                         ON CAST(g.geolocation_zip_code_prefix AS INT) = cp.cep_prefix")

In [17]:
# Verificando os valores missing
geolocalization.select([count(when(isnan(coluna) | col(coluna).isNull(), coluna)).alias(coluna) for coluna in geolocalization.columns]
   ).show()

NameError: name 'count' is not defined

In [ ]:
# Como a coluna geolocation_city_CEP possui valores missing, terei que inputar 
# os valores originais da coluna geolocation_city. Para isso, irei dividir a
# tabela em duas, uma com os valores missing e outra com os valores não missing
# e depois concatená-las

geolocalization_com_missing = geolocalization.filter("geolocation_city IS NULL")
geolocalization = geolocalization.filter("geolocation_city IS NOT NULL")
geolocalization_com_missing = geolocalization_com_missing.withColumn('geolocation_city', col("geolocation_city_original"))
geolocalization = geolocalization.union(geolocalization_com_missing)

In [ ]:
geolocalization.show()

+---------------------------+-------------------+------------------+--------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|   geolocation_lng|geolocation_city_CEP|geolocation_city|geolocation_state|
+---------------------------+-------------------+------------------+--------------------+----------------+-----------------+
|                      01037| -23.54562128115268|-46.63929204800168|           são paulo|       sao paulo|               sp|
|                      01037| -23.54562128115268|-46.63929204800168|           são paulo|       sao paulo|               sp|
|                      01046|-23.546081127035535|-46.64482029837157|           são paulo|       sao paulo|               sp|
|                      01046|-23.546081127035535|-46.64482029837157|           são paulo|       sao paulo|               sp|
|                      01046| -23.54612896641469|-46.64295148361138|           são paulo|       sao paulo|               sp|


## order_items

In [20]:
# Antes de decidi as mudanças, devemos checar as primeiras linhas e o schema
order_items.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.90|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.90|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.00|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.90|        18.14|
+--------------------+-------------+----

In [65]:
order_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- freight_value: string (nullable = true)



Para essa tabela, irei alterar o tipo de dado das colunas ``price`` e 
``freight_value`` para **Float** e ``shipping_limit_date``para **timestamp**.

In [70]:
order_items = spark.sql("SELECT order_id, \
                          order_item_id, \
                          product_id, \
                          seller_id, \
                          CAST(shipping_limit_date AS TIMESTAMP), \
                          CAST(price AS FLOAT), \
                          CAST(freight_value AS FLOAT)  \
                         FROM order_items")

## order_payments

In [71]:
# Vamos checar as primeiras linhas e o schema
order_payments.show(5)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows



In [72]:
order_payments.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: string (nullable = true)
 |-- payment_value: string (nullable = true)



Para essa tabela, irei alterar o tipo de dado das colunas ``payment_sequential``
e ``payment_installments`` para **Int** e ``payment_value`` para **Float**.

In [75]:
order_payments = spark.sql("SELECT order_id, \
                             CAST(payment_sequential AS INT), \
                             payment_type, \
                             CAST(payment_installments AS INT), \
                             CAST(payment_value AS FLOAT)  \
                            FROM order_payments")

## order_reviews

In [77]:
# Vamos checar as primeiras linhas e o schema
order_reviews.show(5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                null|                  null| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                null|                  null| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                null|                  null| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                null|  Recebi bem antes ...| 2017-04-21 00:00:00|   

In [78]:
order_reviews.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



In [38]:
orders_full = spark.sql("WITH geo_sellers AS  \
                            ( \
                            SELECT s.seller_zip_code_prefix, \
                            s.seller_city, \
                            s.seller_id,\
                            s.seller_state, \
                            g.geolocation_lat, \
                            g.geolocation_lng \
                            FROM sellers s  \
                            FULL OUTER JOIN geolocalization g  \
                            ON s.seller_zip_code_prefix = g.geolocation_zip_code_prefix \
                            ),  \
                            item_info AS \
                            ( \
                            SELECT oi.order_id, \
                            oi.order_item_id, \
                            oi.product_id, \
                            oi.seller_id, \
                            oi.shipping_limit_date, \
                            oi.price, \
                            oi.freight_value, \
                            p.product_category_name, \
                            p.product_name_lenght, \
                            p.product_description_lenght,  \
                            p.product_photos_qty, \
                            p.product_weight_g,  \
                            p.product_length_cm, \
                            p.product_height_cm, \
                            p.product_width_cm  \
                            FROM order_items oi  \
                            LEFT JOIN products p  \
                            ON oi.product_id = p.product_id \
                            )  \
                            SELECT o.order_id, \
                            o.customer_id, \
                            ii.product_id,  \
                            o.order_status, \
                            CAST(o.order_estimated_delivery_date AS date), \
                            CAST(o.order_purchase_timestamp AS timestamp), \
                            CAST(o.order_approved_at AS timestamp),  \
                            CAST(o.order_delivered_carrier_date AS timestamp), \
                            CAST(o.order_delivered_customer_date AS timestamp),  \
                            CAST(or.review_score AS int), \
                            LOWER(or.review_comment_title) AS review_comment_title, \
                            LOWER(or.review_comment_message) AS review_comment_message, \
                            CAST(or.review_creation_date AS date), \
                            CAST(or.review_answer_timestamp AS timestamp),  \
                            CAST(op.payment_sequential AS int), \
                            op.payment_type, \
                            CAST(op.payment_installments AS int),  \
                            CAST(op.payment_value AS float), \
                            c.customer_city, \
                            LOWER(c.customer_state) AS customer_state,  \
                            gs.seller_city, \
                            LOWER(gs.seller_state) AS seller_state, \
                            CAST(ii.shipping_limit_date AS timestamp), \
                            CAST(ii.price AS float), \
                            CAST(ii.freight_value AS float), \
                            ii.product_category_name, \
                            CAST(ii.product_name_lenght AS int), \
                            CAST(ii.product_description_lenght AS int),  \
                            CAST(ii.product_photos_qty AS int), \
                            CAST(ii.product_weight_g AS int),  \
                            CAST(ii.product_length_cm AS int), \
                            CAST(ii.product_height_cm AS int), \
                            CAST(ii.product_width_cm AS int),  \
                            CASE  \
                                WHEN c.customer_state IN ('ac', 'am', 'ro', 'rr', 'pa', 'ap', 'to') THEN 'norte'  \
                                WHEN c.customer_state IN ('ma', 'pi', 'ce', 'rn', 'pb', 'pe', 'se', 'al', 'ba') THEN 'nordeste'  \
                                WHEN c.customer_state IN ('mt', 'ms', 'go', 'df') THEN 'centro-oeste'  \
                                WHEN c.customer_state IN ('sp', 'rj', 'es', 'mg') THEN 'sudeste'  \
                                ELSE 'sul'  \
                            END AS customer_region, \
                            CASE  \
                                WHEN gs.seller_state IN ('ac', 'am', 'ro', 'rr', 'pa', 'ap', 'to') THEN 'norte'  \
                                WHEN gs.seller_state IN ('ma', 'pi', 'ce', 'rn', 'pb', 'pe', 'se', 'al', 'ba') THEN 'nordeste'  \
                                WHEN gs.seller_state IN ('mt', 'ms', 'go', 'df') THEN 'centro-oeste'  \
                                WHEN gs.seller_state IN ('sp', 'rj', 'es', 'mg') THEN 'sudeste'  \
                                ELSE 'sul'  \
                            END AS seller_region, \
                            CASE  \
                                WHEN ii.product_category_name IN ('fashion_roupa_masculina', 'fashion_roupa_feminina', 'fashion_roupa_infanto_juvenil', 'fashion_underwear_e_moda_praia', 'fashion_bolsas_e_acessorios', 'fashion_calcados', 'fashion_esporte') THEN 'moda e acessorios'  \
                                WHEN ii.product_category_name IN ('moveis_decoracao', 'moveis_colchao_e_estofado', 'moveis_cozinha_area_de_servico_jantar_e_jardim', 'moveis_quarto', 'moveis_sala', 'utilidades_domesticas', 'cama_mesa_banho', 'casa_construcao', 'casa_conforto', 'casa_conforto_2') THEN 'casa e decoracao'  \
                                WHEN ii.product_category_name IN ('livros_tecnicos', 'livros_importados','livros_interesse_geral') THEN 'livros e educacao'  \
                                WHEN ii.product_category_name IN ('beleza_saude', 'fraldas_higiene', 'perfumaria') THEN 'beleza e saude'  \
                                WHEN ii.product_category_name IN ('brinquedos', 'jogos', 'instrumentos_musicais') THEN 'brinquedos e jogos'  \
                                WHEN ii.product_category_name IN ('alimentos_bebidas', 'bebidas', 'alimentos') THEN 'alimentos e bebidas'  \
                                WHEN ii.product_category_name IN ('artigos_de_festas', 'artigos_de_natal') THEN 'artigos para festas'  \
                                WHEN ii.product_category_name IN ('artes', 'cine_foto', 'artes_e_artesanato') THEN 'artes e artesanato'  \
                                WHEN ii.product_category_name IN ('construcao_ferramentas_construcao', 'construcao_ferramentas_seguranca', 'construcao_ferramentas_jardim', 'construcao_ferramentas_iluminacao', 'construcao_ferramentas_ferramentas') THEN 'construcao e ferramentas'  \
                                WHEN ii.product_category_name IN ('esporte_lazer') THEN 'esporte e lazer'  \
                                ELSE 'outros'  \
                            END AS product_sub_category, \
                            ii.product_weight_g / 1000 AS product_weight_kg, \
                            ii.product_length_cm / 100 AS product_length_m, \
                            ii.product_height_cm / 100 AS product_height_m, \
                            ii.product_width_cm / 100 AS product_width_m, \
                            DAYOFWEEK(o.order_purchase_timestamp) AS order_weekday, \
                            MONTH(o.order_purchase_timestamp) AS order_month, \
                            HOUR(o.order_purchase_timestamp) AS order_hour, \
                            DATEDIFF(o.order_delivered_customer_date, o.order_purchase_timestamp) AS order_delivery_time, \
                            DATEDIFF(o.order_approved_at, o.order_purchase_timestamp) AS order_approval_time, \
                            CASE \
                                WHEN order_weekday = 1 THEN 'sunday'  \
                                WHEN order_weekday = 2 THEN 'monday'  \
                                WHEN order_weekday = 3 THEN 'tuesday'  \
                                WHEN order_weekday = 4 THEN 'wednesday'  \
                                WHEN order_weekday = 5 THEN 'thursday'  \
                                WHEN order_weekday = 6 THEN 'friday'  \
                                ELSE 'saturday'  \
                            END AS order_weekday_name, \
                            CASE \
                                WHEN order_month = 1 THEN 'january'  \
                                WHEN order_month = 2 THEN 'february'  \
                                WHEN order_month = 3 THEN 'march'  \
                                WHEN order_month = 4 THEN 'april'  \
                                WHEN order_month = 5 THEN 'may'  \
                                WHEN order_month = 6 THEN 'june'  \
                                WHEN order_month = 7 THEN 'july'  \
                                WHEN order_month = 8 THEN 'august'  \
                                WHEN order_month = 9 THEN 'september'  \
                                WHEN order_month = 10 THEN 'october'  \
                                WHEN order_month = 11 THEN 'november'  \
                                ELSE 'december'  \
                            END AS order_month_name, \
                            CASE  \
                                WHEN or.review_score <= 3 THEN 'bad'  \
                                ELSE 'good'  \
                            END AS review_score_class \
                            FROM orders o \
                            LEFT JOIN order_reviews or  \
                            ON o.order_id = or.order_id  \
                            LEFT JOIN order_payments op  \
                            ON o.order_id = op.order_id  \
                            LEFT JOIN customers c  \
                            ON o.customer_id = c.customer_id  \
                            LEFT JOIN item_info ii   \
                            ON o.order_id = ii.order_id  \
                            LEFT JOIN geo_sellers gs  \
                            ON ii.seller_id = gs.seller_id")

In [ ]:
# Checando o número de linhas
orders_full.count()

In [ ]:
# Checando a quantidade de colunas
len(orders_full.columns)

In [ ]:
# Buscando e dropando duplicatas
orders_full = orders_full.drop_duplicates()

In [ ]:
# Checando a quantidade de linhas novamente
orders_full.count()

Agora com a junção já feita, vamos passar para a fase de tratamento dos dados.
Como a junção foi feita selecionando as colunas, não há colunas duplicadas no dataset.

### Tratando os dados
Nesta etapa, passarei de coluna a coluna tratando o dado conforme necessário.

In [ ]:
# Checando as colunas
orders_full.columns

#### order_Status


In [ ]:
# Checando os valores
orders_full.select('order_status').distinct().collect()

A coluna em questão não demonstra ter erros, onde os valores estão na mesma case
 e sem inconsistências. Além disso, o tipo do dado é **string**, o que 
 corresponde com os valores da coluna.

### order_estimated_delivery_date

In [ ]:
orders_full.select('order_estimated_delivery_date').show()

Como podemos ver, os valores dessa coluna se referem a data estimada da entrega, sem hora e nem minutos. Por isso, será transformada em apenas **data** e não **timestamp**.

In [ ]:
# Alterando o tipo para data
orders_full = orders_full.withColumn("order_estimated_delivery_date", col("order_estimated_delivery_date").cast('date'))


### order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date
Aqui 4 colunas serão tratadas juntas, pois demandam o mesmo tratamento, **string** para **timestamp**.

In [ ]:
# Criando uma lista com as colunas
colunas = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']

In [ ]:
# Realizando a transformação em cada coluna
for coluna in colunas:
    orders_full = orders_full.withColumn(coluna, to_timestamp(coluna))

### review_score
A princípio essa coluna parece estar com o tipo errado, vamos verificar os valores:

In [ ]:
# Checando os valores distintos
orders_full.select('review_score').distinct().show()

In [ ]:
# Alterando o tipo para data
orders_full = orders_full.withColumn("review_score", col("review_score").cast('int'))


### review_comment_title

In [ ]:
# Checando os valores 
orders_full.select('review_comment_title').show()

Aqui o formato parece estar correto, mas irei limpar o texto, retirando pontuações e normalizando o texto (mesma case).

In [ ]:
# regex para remover pontuações
re_special = "[^\w\s]"

In [ ]:
# Criando funções para remover caracteres especiais
func = lambda titulo: re.sub(re_special, ' ', str(titulo).strip())
func_udf = udf(func, StringType())

In [ ]:
# Aplicando a função
orders_full = orders_full.withColumn('review_comment_title', func_udf(lower('review_comment_title')))

Agora irei criar tokens a partir do texto.

In [ ]:
# Criando um tokenizador
tokenizer = Tokenizer(inputCol="review_comment_title", outputCol="review_comment_title_tokens")

# Criando nova coluna com os tokens
orders_full = tokenizer.transform(orders_full)

In [ ]:
# Checando os valores 
orders_full.select('review_comment_title').show()

In [ ]:
# Checando os valores 
orders_full.select('review_comment_title_tokens').show()

### review_comment_message

In [ ]:
# Checando os valores
orders_full.select('review_comment_message').show()

Aqui eu usarei o mesmo tratamento da coluna passada, removendo pontuações e 
deixando o texto na mesma case.

In [ ]:
# Aplicando a função
orders_full = orders_full.withColumn('review_comment_message', func_udf(lower('review_comment_message')))

Por fim, irei criar os tokens.

In [ ]:
# Criando o tokenizer
tokenizer = Tokenizer(inputCol="review_comment_message", outputCol="review_comment_message_tokens")

# Criando nova coluna com os tokens
orders_full = tokenizer.transform(orders_full)

In [ ]:
# Checando os valores 
orders_full.select('review_comment_message').show()

In [ ]:
# Checando os valores 
orders_full.select('review_comment_message_tokens').show()

### review_creation_date

In [ ]:
# Checando os valores 
orders_full.select('review_creation_date').show()

Como podemos ver, os valores dessa coluna se referem a data de criação da review, sem hora e nem minutos. Por isso, será transformada em apenas **data** e não **timestamp**.

In [ ]:
# Alterando o tipo de dado
orders_full = orders_full.withColumn('review_creation_date', col('review_creation_date').cast('date'))

### review_answer_timestamp

In [ ]:
# Checando os valores
orders_full.select('review_answer_timestamp').show()

Desta vez os dados possuem data e hora, fazendo com que seja necessário transforma-los para o formato **timestamp**.

In [ ]:
# Alterando o tipo de dado
orders_full = orders_full.withColumn('review_answer_timestamp', to_timestamp('review_answer_timestamp'))

### payment_sequential

In [ ]:
# Verificando os valores únicos
orders_full.select('payment_sequential').distinct().show()

O dado aparenta ser numerico, representando o número de pagamentos.

In [ ]:
# Alterando o tipo de dado
orders_full = orders_full.withColumn('payment_sequential', col('payment_sequential').cast('int'))

### payment_type

In [ ]:
# Checando os valores
orders_full.select('payment_type').distinct().show()

Não aparenta haver inconsistências nos valores e nem no tipo do dado. Por isso, não irei aplicar tatamento nesta coluna.

### payment_installments

In [ ]:
# Checando os valores distintos
orders_full.select('payment_installments').distinct().show()

A coluna aparenta representar o número de parcelas. Embora não apresente 
inconsistências nos valores, ainda é necessário aplicar uma alteração de tipo
de dado.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('payment_installments', col('payment_installments').cast('int'))

### payment_value

In [ ]:
# Checando os valores distintos
orders_full.select('payment_value').distinct().show()

A coluna represaenta o valor de pagamento. O tipo de dado deve ser alterado para float.

In [ ]:
# Alterando o tipo de dado
orders_full = orders_full.withColumn('payment_value', col('payment_value').cast('float'))

### customer_city

In [ ]:
# Checando os valores distintos
orders_full.select('customer_city').distinct().show(20)

Os dados da coluna já estão na mesma case, sem pontuações e possui o formato 
correto, tornando desnecessário a aplicação de tratamento na mesma.

### customer_state

In [ ]:
# Checando os valores distintos
orders_full.select('customer_state').distinct().collect()

Os dados da coluna já estão na mesma case, sem pontuações e possui o formato 
correto, tornando desnecessário a aplicação de tratamento na mesma.

### seller_city

In [ ]:
# Checando os valores distintos
orders_full.select('seller_city').distinct().show()

Os dados da coluna já estão na mesma case, sem pontuações e possui o formato 
correto, tornando desnecessário a aplicação de tratamento na mesma.

### seller_state

In [ ]:
# Checando os valores distintos
orders_full.select('seller_state').distinct().collect()

Os dados da coluna já estão na mesma case, sem pontuações e possui o formato 
correto, tornando desnecessário a aplicação de tratamento na mesma.

### shipping_limit_date

In [ ]:
# Checando os valores 
orders_full.select('shipping_limit_date').show()

Como mostrado no schema, os dados estão no formato de **string**. Aqui, irei
alterar o tipo para timestamp, pois possui data e hora.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('shipping_limit_date', to_timestamp('shipping_limit_date'))

### price

In [ ]:
# Checando a coluna
orders_full.select('price').show()

Aqui tenho números com centavos, por isso, será alterado para **float**.

In [ ]:
orders_full = orders_full.withColumn('price', col('price').cast('float'))

### freight_value

In [ ]:
# Checando a coluna
orders_full.select('freight_value').show()

Aqui tenho números com centavos, por isso, será alterado para **float**.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('freight_value', col('freight_value').cast('float'))

### product_category_name

In [ ]:
# Checando a coluna
orders_full.select('product_category_name').distinct().show()

Os dados da coluna já estão na mesma case, sem pontuações e possui o formato 
correto, tornando desnecessário a aplicação de tratamento na mesma.

### product_name_lenght

In [ ]:
# Checando a coluna 
orders_full.select('product_name_lenght').show()

A coluna contém um dado com o tamanho do nome do produto. Como esse número não
pode possuir casas decimais, será convertido para **int**.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('product_name_lenght', col('product_name_lenght').cast('int'))

### product_description_lenght

In [ ]:
# Checando a coluna
orders_full.select('product_description_lenght').show()

A coluna contém um dado com o tamanho da descrição do produto. Como esse número não
pode possuir casas decimais, será convertido para **int**.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('product_description_lenght', col('product_description_lenght').cast('int'))

### product_photos_qty

In [ ]:
# Checando a coluna
orders_full.select('product_photos_qty').show()

A coluna contém um dado com a quantidade de fotos do produto. Como esse número não
pode possuir casas decimais, será convertido para **int**.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('product_photos_qty', col('product_photos_qty').cast('int'))

### product_weight_g

In [ ]:
# Checando a coluna
orders_full.select('product_weight_g').show()

A coluna contém um dado com o peso do produto em gramas. Como esse número não
pode possuir casas decimais, será convertido para **int**.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('product_weight_g', col('product_weight_g').cast('int'))

### product_length_cm

In [ ]:
# Checando a coluna
orders_full.select('product_length_cm').show()

A coluna contém um dado com o comprimento do produto em centímetros. Como esse número não
pode possuir casas decimais, será convertido para **int**.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('product_length_cm', col('product_length_cm').cast('int'))

### product_height_cm

In [ ]:
# Checando a coluna
orders_full.select('product_height_cm').show()

A coluna contém um dado com a altura do produto em centímetros. Como esse número não
pode possuir casas decimais, será convertido para **int**.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('product_height_cm', col('product_height_cm').cast('int'))

### product_width_cm

In [ ]:
# Checando a coluna
orders_full.select('product_width_cm').show()

A coluna contém um dado com a largura do produto em centímetros. Como esse número não
pode possuir casas decimais, será convertido para **int**.

In [ ]:
# Alterando o tipo do dado
orders_full = orders_full.withColumn('product_width_cm', col('product_width_cm').cast('int'))

#### Criando novas features

Agora que a tabela já parece estar boa, irei criar uma novas features. A primeira 
será criada a partir das colunas ``customer_state`` e ``seller_state``, criando categorias para os 
estados, onde cada estado vai receber a região em que se localiza no Brasil.

In [ ]:
# Criando lista de regiões  
estados_norte = ['AC', 'AM', 'RO', 'RR', 'AP', 'TO', 'PA']
estados_nordeste = ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'BA', 'SE']
estados_centro = ['MT', 'MS', 'DF', 'GO']
estados_sudeste = ['MG', 'SP', 'ES', 'RJ']
estados_sul = ['PR', 'SC', 'RS']

# Criando função 
func = lambda estado: 'NORTE' if estado in estados_norte else \
        ('NORDESTE' if estado in estados_nordeste else \
        ('CENTRO-OESTE' if estado in estados_centro else \
        ('SUDESTE' if estado in estados_sudeste else 'SUL')))

func_udf = udf(func, StringType())

In [ ]:
# Aplicando a função
orders_full = orders_full.withColumn("customer_region", func_udf("customer_state"))
orders_full = orders_full.withColumn("seller_region", func_udf("seller_state"))

Agora, irei criar uma nova coluna para resumir as categorias da coluna
``product_category_name``.

In [ ]:
# Criando novas categorias
eletronicos_e_tecnologia = ['pcs', 'pc_gamer', 'tablets_impressao_imagem', 
                            'telefonia_fixa', 'telefonia', 
                            'informatica_acessorios', 'eletronicos', 'audio', 
                            'consoles_games', 'dvds_blu_ray']

moda_e_acessorios = ['fashion_roupa_masculina', 'fashion_roupa_feminina',
                     'fashion_roupa_infanto_juvenil',
                     'fashion_underwear_e_moda_praia',
                     'fashion_bolsas_e_acessorios', 'fashion_calcados',
                     'fashion_esporte']

casa_e_decoracao = ['moveis_decoracao', 'moveis_colchao_e_estofado', 
                    'moveis_cozinha_area_de_servico_jantar_e_jardim', 
                    'moveis_quarto', 'moveis_sala', 'utilidades_domesticas', 
                    'cama_mesa_banho', 'casa_construcao', 'casa_conforto', 
                    'casa_conforto_2']

livros_e_educacao = ['livros_tecnicos', 'livros_importados', 
                     'livros_interesse_geral']

beleza_e_saude = ['beleza_saude', 'fraldas_higiene', 'perfumaria']

brinquedos_e_jogos = ['brinquedos', 'jogos', 'instrumentos_musicais']

alimentos_e_bebidas = ['alimentos_bebidas', 'bebidas', 'alimentos']

artigos_para_festas = ['artigos_de_festas', 'artigos_de_natal']

arte_e_artesanato = ['artes', 'cine_foto', 'artes_e_artesanato']

ferramentas_e_construcao = ['construcao_ferramentas_construcao', 
                            'construcao_ferramentas_seguranca', 
                            'construcao_ferramentas_jardim', 
                            'construcao_ferramentas_iluminacao', 
                            'construcao_ferramentas_ferramentas']

esporte_e_lazer = ['esporte_lazer']

outros = ['cool_stuff', 'flores', 'industria_comercio_e_negocios', 
          'malas_acessorios', 'seguros_e_servicos', 'market_place', 
          'relogios_presentes', 'papelaria', 'climatizacao', 
          'sinalizacao_e_seguranca', 'agro_industria_e_comercio', 
          'cds_dvds_musicais', 'musica', 'eletroportateis']

In [ ]:
# Criando a função
func = lambda categoria: 'eletronicos e tecnologia' if categoria in eletronicos_e_tecnologia else \
       ('moda e acessorios' if categoria in moda_e_acessorios else \
       ('casa e decoracao' if categoria in casa_e_decoracao else \
       ('livros e educacao' if categoria in livros_e_educacao else \
       ('beleza e saude' if categoria in beleza_e_saude else \
       ('brinquedos e jogos' if categoria in brinquedos_e_jogos else \
       ('alimentos_e_bebidas' if categoria in alimentos_e_bebidas else \
       ('artigos para festas' if categoria in artigos_para_festas else \
       ('arte e artesanato' if categoria in arte_e_artesanato else \
       ('ferramentas e construcao' if categoria in ferramentas_e_construcao else \
       ('esporte e lazer' if categoria in esporte_e_lazer else 'outros'))))))))))
       
func_udf = udf(func, StringType())


In [ ]:
# Criando nova coluna
orders_full = orders_full.withColumn('sub_category_product_name', func_udf('product_category_name'))

Como novas colunas, irei criar novas unidades de medida a partir  das 
unidades das colunas ``product_weight_g``, ``product_length_cm``, 
``product_height_cm`` e ``product_width_cm``.

In [ ]:
# Alterando a unidade de medida das colunas
orders_full = orders_full.withColumn('product_weight_kg', col('product_weight_g')/1000)
orders_full = orders_full.withColumn('product_length_m', col('product_length_cm')/100)
orders_full = orders_full.withColumn('product_height_m', col('product_height_cm')/100)
orders_full = orders_full.withColumn('product_width_m', col('product_width_cm')/100)


E para finalizar, irei extrair novas colunas a partir dos dados que representam datas.

In [ ]:
# Obtendo o dia da semana da compra
orders_full = orders_full.withColumn('order_day_of_week', dayofweek('order_purchase_timestamp'))

# Criando uma função para obter o nome do dia da semana
func = lambda dia: 'sunday' if dia == 1 else  \
        ('monday' if dia == 2 else  \
        ('tuesday' if dia == 3 else  \
        ('wednesday' if dia == 4 else  \
        ('thursday' if dia == 5 else  \
        ('friday' if dia == 6 else 'saturday')))))

func_udf = udf(func, StringType())

# Aplicando a função
orders_full = orders_full.withColumn('order_name_day_of_week', func_udf('order_day_of_week'))

In [ ]:
# Obtendo o mês da compŕa
orders_full = orders_full.withColumn('order_month', month('order_purchase_timestamp'))

# Criando uma função para nomear os meses
func = lambda mes: 'january' if mes == 1 else  \
        ('february' if mes == 2 else  \
        ('march' if mes == 3 else  \
        ('april' if mes == 4 else  \
        ('may' if mes == 5 else  \
        ('june' if mes == 6 else  \
        ('july' if mes == 7 else  \
        ('august' if mes == 8 else  \
        ('september' if mes == 9 else  \
        ('october' if mes == 10 else  \
        ('november' if mes == 11 else 'december'))))))))))
        
func_udf = udf(func, StringType())

# Aplicando a função
orders_full = orders_full.withColumn('order_name_month', func_udf('order_month'))

In [ ]:
# Obtendo a hora da compra
orders_full = orders_full.withColumn('order_hour', hour('order_purchase_timestamp'))

In [ ]:
# Obtendo o tempo de processamento do pedido
orders_full = orders_full.withColumn('order_processing_days', datediff('order_approved_at', 'order_purchase_timestamp'))

In [ ]:
# Calculando a diferença de dias entre a data do pedido e a data de entrega
orders_full = orders_full.withColumn('order_delivery_days', datediff('order_delivered_customer_date', 'order_purchase_timestamp'))

## Salvando os dados

In [ ]:
# Criando o db
db_connection = mysql.connector.connect(user="root", password="123456", host="localhost", auth_plugin='mysql_native_password')
db_cursor = db_connection.cursor()
db_cursor.execute("CREATE DATABASE IF NOT EXISTS dbTeste;")

In [ ]:
mysql_url = "jdbc:mysql://localhost:3306/dbTeste"
mysql_properties = {
    "user": "root",
    "password": "123456",
    "driver": "com.mysql.jdbc.Driver"
}

#orders_full.write.jdbc(mysql_url, "tableteste", mode="overwrite", properties=mysql_properties)
orders_full.write \
    .format("jdbc") \
    .option("url", mysql_url) \
    .option("dbtable", "teste") \
    .mode("overwrite") \
    .options(**mysql_properties) \
    .save()



In [ ]:

# Write to MySQL Table
orders_full.write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", "jdbc:mysql://localhost:3306/dbTeste") \
  .option("dbtable", "dbTeste") \
  .option("user", "root") \
  .option("password", "123456") \
  .save('append')
